## Module imports

In [54]:
import keras
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense
from keras.models import Model

from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

from keras.models import model_from_json

import cv2
import numpy as np


## Parameters for training

In [ ]:
# Number of epochs
epochs = 50

# Batch size
batch_size = 16

# Number of categories
classes = <your classes>

# Image size
image_size = (<your size>,<your size>)

# Number of training and validation samples
number_of_images_training = <your number of training images>
number_of_images_validation = <your number of validation images>

# Steps per epoch and validation steps per epoch
steps_per_epoch = number_of_images // batch_size
validation_steps = number_of_images // batch_size


## Image augmentations

### Traning augmentations

In [ ]:
# Augment the data during training
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
rescale=1./255)

### Validation set normalization

In [ ]:
validation_datagen = ImageDataGenerator(
    rescale=1./255)

## Initialize VGG16 base model and weights

In [ ]:
base_model= VGG16(weights='imagenet', include_top=False, input_shape=(image_size,3))

## Adding top model to base model

In [ ]:
top_model = Flatten()(base_model.output)
top_model = Dense(256, activation='relu')(top_model)
top_model = Dense(classes, activation='softmax')(top_model)

## Instantiate the functional model object 

In [ ]:
model = Model(inputs=base_model.input, outputs=top_model)

## Compile model

In [ ]:
# 'binary_crossentropy' for 2 classes, 'categorical_crossentropy' > 2 classes
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

## Lock all convolution and pooling layers

In [ ]:
for layer in model.layers[:20]:
    layer.trainable = False

In [ ]:
model.summary()

## Load datasets using data generators

In [ ]:
train_generator = train_datagen.flow_from_directory(
        # Add training directory
        'data/train',
        target_size=(150, 150),
        batch_size=batch_size,
    
        # 'binary' for 2 classes, 'categorical' > 2 classes
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        # Add validation directory
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
    
        # 'binary' for 2 classes, 'categorical' > 2 classes
        class_mode='categorical')


## Create class labels

In [50]:
label_map = (train_generator.class_indices)

# Invert the dictionary
inverted_label_map = {v:k for k,v in label_map.items()}
print(inverted_label_map)

{0: 'angus', 1: 'highland', 2: 'holstein_friesian'}


## Train the model

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_steps)

## Save the model architecture in JSON

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

## Save model weights

In [ ]:
model.save_weights('your_model.h5')

## Load saved model and saved architecture

In [52]:
# Load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Load saved weights into new model
loaded_model.load_weights("new_model.h5")

## Try to predict an image using the trained and saved model

In [57]:
# path to image for prediction
img_input = "path_to_image"

img_for_prediction = cv2.imread(img_input)

# Resize image to the same image as training image size
resized_image = cv2.resize(img_for_prediction, dsize=(image_size), interpolation=cv2.INTER_CUBIC)

# Reshape (image_size,3) to (1,image_size,3)
img_for_prediction = resized_image[np.newaxis,...]

# Predict
prediction = loaded_model.predict(img_for_prediction)

# Print image prediction
pred_label = prediction.argmax(axis=-1)
print("Prediction: ", inverted_label_map[pred_label[0]])

Prediction:  holstein_friesian
